In [19]:
from datetime import datetime
from datetime import date
from subgrounds.subgraph import SyntheticField
from subgrounds.subgrounds import Subgrounds
import pandas as pd

In [20]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

In [21]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
)

In [22]:
spacename = input('spacename plz?')
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=10000,
  where=[
    snapshot.Proposal.space == spacename, ##'fuse.eth',
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)

In [23]:
proposals_snapshots = sg.query_df([
    proposals.title,
    proposals.id,
    proposals.body,
    proposals.scores,
    proposals.scores_total
])

In [24]:
proposals_choices = sg.query(proposals.choices)

TypeError: 'FieldPath' object is not iterable

In [22]:
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_1', 'option_2', 'option_3', 'option_4', 'option_5','option_6','option_7','option_8','option_9','option_10','option_11','option_12','option_13','option_14','option_15','option_16','option_17','option_18','option_19','option_20','option_21','option_22'])

NameError: name 'proposals_choices' is not defined

In [23]:
olympus_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)

NameError: name 'proposals_choices' is not defined

In [24]:
##let's view the output just to make sure
olympus_governance_view

NameError: name 'olympus_governance_view' is not defined

In [14]:
file = input('Selet a folder') ##enter your file path here - the file is in the repo "summary_stats.csv".
path =file+'/'+spacename+'_proposals_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(olympus_governance_view))+'_proposals.csv'
voting_snapshots_list.to_csv(path, index = False)

NameError: name 'spacename' is not defined

In [268]:
total_proposals = len(olympus_governance_view)
total_proposals

145

In [269]:
proposal_id = olympus_governance_view.iloc[0,1]

In [270]:
print(proposal_id)

0xd75cab98b9143d7a16504dad01a3322ce6855369ff6ca8d7b1685a4ac694a217


In [271]:

vote_tracker = snapshot.Query.votes(
orderBy = 'created',
orderDirection='desc',
first=10000,
where=[
  snapshot.Vote.proposal == proposal_id
]
)

In [272]:
voting_snapshots_list = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice,
    vote_tracker.vp
])

In [273]:
voting_snapshots_list

,votes_id,votes_voter,votes_created,votes_choice,votes_vp
0,0x8281ce505c29b6b019961f6e0e6e14fde7c5bc15d50f...,0x97b90FBc8904F861F76CB06BFa0A465b72C5E662,1650667132,1,1.185496e+05
1,0xa5d685dabf23e7ae3a60954e330a6106c26b41b40e5f...,0xC2634B9d0A0cFb756cDed34656E09828072A8a9B,1650628049,1,6.096007e+04
2,0xca7d2b7344087e3af22224271a927c3e6b857d8b0a34...,0x29687E2a2b38bbc1f4f9069f1f24F13a3c83d27D,1650568499,1,2.702278e+04
3,0x5a1e05b9a22922069534917a5a0b00cc4cf32dfb17a5...,0x1ccb144B700EC726d37Db38c617E154De6d9c0d0,1650420674,1,4.523094e+02
4,0x7d5245e24f681f0dfe15ca64913f29982630a9f79213...,0x839395e20bbB182fa440d08F850E6c7A8f6F0780,1650290002,1,2.173455e+03
5,0x0d63f77e2a10c397612f55858cc3a6f2510cdcb3c947...,0x8639F60e8F9b984eaf62B0bd89F62D6161e182F9,1650283481,1,2.545218e+06


In [25]:
voting_snapshots_list['Proposal'] = proposal_id
voting_snapshots_list

NameError: name 'proposal_id' is not defined

In [275]:
x=0
while x <total_proposals:
##print(olympus_governance_view.iloc[2,1])
    proposal_id = olympus_governance_view.iloc[x,1]

    vote_tracker = snapshot.Query.votes(
    orderBy = 'created',
    orderDirection='desc',
    first=10000,
    where=[
      snapshot.Vote.proposal == proposal_id
    ]
    )
    voting_snapshots = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice,
    vote_tracker.vp
    ])

    voting_snapshots['Proposal'] = proposal_id
    voting_snapshots_list=pd.concat([voting_snapshots_list, voting_snapshots])
    ##voting_snapshots['Proposal'] = proposal_id

#    pd.concat([voting_snapshots_list, voting_snapshots])
    x=x+1

In [276]:
len(voting_snapshots_list)

4389

In [277]:
path =file+'/'+spacename+'_voting_snapshots_list_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(olympus_governance_view))+'.csv'
voting_snapshots_list.to_csv(path, index = False)